In [1]:
# 미국꺼
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import mysql.connector

# MySQL에 연결하여 데이터를 삽입하는 함수
def insert_news_to_db(state, url, title, image_url, content):
    try:
        # MySQL 연결
        connection = mysql.connector.connect(
            user='yangjinan',
            password='ALsk85200!!',
            host='hanaex.mysql.database.azure.com',
            database='my_database',
            port=3306
        )
        cursor = connection.cursor()

        # SQL INSERT 쿼리
        insert_query = """
        INSERT INTO fo_news_table (state, url, title, image_url, content)
        VALUES (%s, %s, %s, %s, %s)
        """
        data = (state, url, title, image_url, content)
        cursor.execute(insert_query, data)
        
        # 트랜잭션 커밋
        connection.commit()
        print(f"Data inserted successfully: {title}")

    except mysql.connector.Error as error:
        print(f"Failed to insert data: {error}")

    finally:
        # 커넥션 및 커서 종료
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed")

# 크롬 드라이버 설정
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# 1. Investing.com 뉴스 검색 결과 페이지에 접속
driver.get("https://www.investing.com/search/?q=USD&tab=news")
time.sleep(3)  # 페이지 로딩을 기다림

# 2. 검색 결과에서 뉴스 항목들을 순회
news_links = set()  # 중복 링크를 방지하기 위해 set 사용
articles_section = driver.find_element(By.CSS_SELECTOR, "#fullColumn > div > div:nth-child(6) > div.searchSectionMain > div")
article_elements = articles_section.find_elements(By.CSS_SELECTOR, "div > a")

# 각 뉴스의 링크 수집
for article in article_elements:
    news_links.add(article.get_attribute("href"))  # 중복되지 않게 set에 추가

# 3. 수집한 뉴스 링크에 접속하여 title, content, image 크롤링 및 DB에 저장
for link in news_links:
    driver.get(link)
    time.sleep(2)  # 페이지 로딩을 기다림

    try:
        # 타이틀 크롤링
        title = driver.find_element(By.CSS_SELECTOR, "#articleTitle").text
        print("Title:", title)

        # 이미지 크롤링
        image = driver.find_element(By.CSS_SELECTOR, "#__next > div.md\\:relative.md\\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\\[\\#232526\\].antialiased.xl\\:container.sm\\:px-6.md\\:grid-cols-\\[1fr_72px\\].md\\:gap-6.md\\:px-7.md\\:pt-10.md2\\:grid-cols-\\[1fr_420px\\].md2\\:gap-8.md2\\:px-8.xl\\:mx-auto.xl\\:gap-10.xl\\:px-10 > div.min-w-0 > div > div > div.relative.flex.flex-col > div.mb-5.mt-4.sm\\:mt-8.md\\:mb-8.relative.h-\\[294px\\].w-full.overflow-hidden.bg-\\[\\#181C21\\].sm\\:h-\\[420px\\].xl\\:h-\\[441px\\] > img").get_attribute("src")
        print("Image URL:", image)

        # 콘텐츠 크롤링
        content = driver.find_element(By.CSS_SELECTOR, "#article > div").text
        print("Content:", content)

        # 데이터베이스에 저장 (state는 "USD"로 고정)
        insert_news_to_db("USD", link, title, image, content)

    except Exception as e:
        print("Error occurred:", e)

# 4. 드라이버 종료
driver.quit()

Title: Saudi Arabia stocks lower at close of trade; Tadawul All Share down 1.58%
Image URL: https://i-invdn-com.investing.com/news/LYNXNPEE310YB_L.jpg
Content: Investing.com – Saudi Arabia stocks were lower after the close on Sunday, as losses in the Industrial Investment, Real Estate Development and Transport sectors led shares lower.
At the close in Saudi Arabia, the Tadawul All Share fell 1.58%.
The best performers of the session on the Tadawul All Share were Al Baha Investment and Development Company SJSC (TADAWUL:4130), which rose 7.14% or 0.02 points to trade at 0.30 at the close. Meanwhile, United Wire Factories Company (TADAWUL:1301) added 3.32% or 1.00 points to end at 31.10 and Kingdom Holding Company (TADAWUL:4280) was up 2.23% or 0.24 points to 10.98 in late trade.
The worst performers of the session were Saudi Ceramic Co. (TADAWUL:2040), which fell 7.26% or 2.25 points to trade at 28.75 at the close. Al ELM Information Security Company CJSC (TADAWUL:7203) declined 6.79% or

Data inserted successfully: MYT Netherlands Parent B.V. (Mytheresa) and Richemont sign agreement for Mytheresa to acquire YOOX NET-A-PORTER (YNAP) to create leading, global, multi-brand digital luxury group in exchan
MySQL connection closed
Title: Spain stocks higher at close of trade; IBEX 35 up 0.49%
Image URL: https://i-invdn-com.investing.com/news/LYNXMPEB1M0J2_L.jpg
Content: Investing.com – Spain stocks were higher after the close on Monday, as gains in the Financial Services & Real Estate, Consumer Goods and Building & Construction sectors led shares higher.
At the close in Madrid, the IBEX 35 added 0.49%.
The best performers of the session on the IBEX 35 were Naturgy Energy Group SA (BME:NTGY), which rose 1.55% or 0.36 points to trade at 23.58 at the close. Meanwhile, Ferrovial (BME:FER) added 1.53% or 0.56 points to end at 37.10 and Banco Santander (BME:SAN) was up 1.53% or 0.07 points to 4.55 in late trade.
The worst performers of the session were Cellnex Telecom SA (BME:CLNX)

Data inserted successfully: France stocks higher at close of trade; CAC 40 up 0.46%
MySQL connection closed
Title: AHT stock touches 52-week low at $0.68 amid market challenges
Image URL: https://i-invdn-com.investing.com/news/news_headline_rolled_69x52._800x533_L_1419494221.jpg
Content: Ashford Hospitality Trust Inc (NYSE:AHT) stock has hit a 52-week low, trading at $0.68, as the company faces ongoing market headwinds. This latest price point reflects a significant downturn for the real estate investment trust, which specializes in the hospitality industry. Over the past year, AHT has seen its stock value decrease by a staggering 69.52%, indicating a tough period for the company amidst a challenging economic environment. Investors are closely monitoring the stock's performance as it navigates through these turbulent times.
In other recent news, Ashford (NYSE:AINC) Hospitality Trust, Inc. has been actively implementing strategies to enhance its financial and operational performance. Th

Data inserted successfully: Southeast Airport Group files routine 6-K with SEC
MySQL connection closed
Title: Poland stocks higher at close of trade; WIG30 up 0.13%
Image URL: https://i-invdn-com.investing.com/news/Warsaw%20Stock%20Exchange_800x533_L_1430990997.jpg
Content: Investing.com – Poland stocks were higher after the close on Monday, as gains in the Information Technology, Media and Construction sectors led shares higher.
At the close in Warsaw, the WIG30 rose 0.13%.
The best performers of the session on the WIG30 were Dino Polska SA (WA:DNP), which rose 5.85% or 19.90 points to trade at 359.90 at the close. Meanwhile, Asseco Poland SA (WA:ACPP) added 5.30% or 4.45 points to end at 88.35 and Eurocash SA (WA:EUR) was up 4.39% or 0.41 points to 9.74 in late trade.
The worst performers of the session were Enea SA (WA:ENAE), which fell 5.66% or 0.62 points to trade at 10.34 at the close. Cyfrowy Polsat SA (WA:CPS) declined 3.63% or 0.46 points to end at 12.07 and Polski Koncern Naf

Title: Netherlands stocks higher at close of trade; AEX up 0.11%
Image URL: https://i-invdn-com.investing.com/news/LYNXMPED0C0KP_L.jpg
Content: Investing.com – Netherlands stocks were higher after the close on Monday, as gains in the Oil & Gas, Telecoms and Consumer Goods sectors led shares higher.
At the close in Amsterdam, the AEX rose 0.11%.
The best performers of the session on the AEX were Shell PLC (AS:SHEL), which rose 1.83% or 0.57 points to trade at 31.71 at the close. Meanwhile, ING Groep NV (AS:INGA) added 0.94% or 0.15 points to end at 16.16 and Koninklijke KPN NV (AS:KPN) was up 0.83% or 0.03 points to 3.65 in late trade.
The worst performers of the session were ASM International NV (AS:ASMI), which fell 2.10% or 12.40 points to trade at 576.80 at the close. BE Semiconductor Industries NV (AS:BESI) declined 1.47% or 1.65 points to end at 110.95 and ASML Holding NV (AS:ASML) was down 1.31% or 9.90 points to 747.60.
Falling stocks outnumbered advancing ones on the Amsterdam 

...

Title: Belgium stocks lower at close of trade; BEL 20 down 0.65%
Image URL: https://i-invdn-com.investing.com/news/LYNXNPEC1R0G3_L.jpg
Content: Investing.com – Belgium stocks were lower after the close on Monday, as losses in the Basic Materials, Consumer Services and Utilities sectors led shares lower.
At the close in Brussels, the BEL 20 lost 0.65%.
The best performers of the session on the BEL 20 were Galapagos NV (AS:GLPG), which rose 6.69% or 1.74 points to trade at 27.74 at the close. Meanwhile, Proximus NV (EBR:PROX) added 1.27% or 0.09 points to end at 6.78 and Ageas (EBR:AGES) was up 0.25% or 0.12 points to 48.22 in late trade.
The worst performers of the session were Warehouses de Pauw Comm VA (EBR:WDPP), which fell 1.76% or 0.42 points to trade at 23.38 at the close. Solvay SA (EBR:SOLB) declined 1.57% or 0.59 points to end at 36.92 and Anheuser Busch Inbev SA NV (EBR:ABI) was down 1.36% or 0.82 points to 59.38.
Falling stocks outnumbered advancing ones on the Brussels Stock

Data inserted successfully: Ferrari N.V.: Periodic Report on the Buyback Program
MySQL connection closed
Title: TD Cowen confident in Acurate approval, reiterates Buy on Boston Scientific stock
Image URL: https://i-invdn-com.investing.com/news/international_newspapers_69x52._800x533_L_1419494241.jpg
Content: On Monday, TD Cowen reaffirmed its Buy rating on shares of Boston Scientific Corporation (NYSE:BSX) and maintained the price target of $86.00.
The firm's outlook remains positive regarding the Acurate IDE trial results, which are anticipated to support the device's approval in the United States. The analyst expressed confidence in the forthcoming results, suggesting that the revised timeline for releasing the trial data indicates a diligent effort by Boston Scientific to expedite the presentation, often a harbinger of favorable results.
The shift in the release schedule for the Acurate IDE trial results to October 2024, previously set for March 2025, was highlighted by TD Cowen as 

Title: Meihua regains compliance with Nasdaq bid price rule
Image URL: https://i-invdn-com.investing.com/news/news_six_pile_69x52._800x533_L_1419494215.jpg
Content: YANGZHOU, China - Meihua International Medical Technologies Co., Ltd. (Nasdaq: MHUA), a manufacturer of disposable medical devices, has announced that it has regained compliance with Nasdaq's minimum bid price requirement. The company received a notification on Monday from the Nasdaq Stock Market's Listing Qualifications Department, confirming that its ordinary shares have maintained a minimum bid price of $1.00 or more per share over the last 21 consecutive business days, from September 4, 2024, to October 2, 2024.
The compliance milestone follows a previous notification on April 2, 2024, indicating that Meihua's shares had fallen below the $1.00 threshold required by Nasdaq's Listing Rule 5450(a)(1) over a 30-day period. The recent improvement in share price has resolved this issue, and the matter is now considered closed

Title: Canada stocks lower at close of trade; S&P/TSX Composite down 0.25%
Image URL: https://i-invdn-com.investing.com/news/LYNXNPEC180BO_L.jpg
Content: Investing.com – Canada stocks were lower after the close on Monday, as losses in the Utilities, Healthcare and IT sectors led shares lower.
At the close in Toronto, the S&P/TSX Composite fell 0.25%.
The best performers of the session on the S&P/TSX Composite were Canadian Natural Resources Ltd (TSX:CNQ), which rose 3.30% or 1.59 points to trade at 49.81 at the close. Meanwhile, MDA Ltd (TSX:MDA) added 3.12% or 0.60 points to end at 19.81 and Imperial Oil Ltd (TSX:IMO) was up 3.04% or 3.19 points to 107.98 in late trade.
The worst performers of the session were IAMGold Corporation (TSX:IMG), which fell 9.50% or 0.65 points to trade at 6.19 at the close. Fortuna Mining Corp (TSX:FVI) declined 8.57% or 0.57 points to end at 6.08 and Emera Incorporated (TSX:EMA) was down 4.97% or 2.59 points to 49.55.
Falling stocks outnumbered advancing 

Title: Trinity Biotech advances next-gen glucose monitor
Image URL: https://i-invdn-com.investing.com/news/World_News_9_800x533_L_1420026261.jpg
Content: DUBLIN - Trinity Biotech plc (NASDAQ:TRIB), a biotechnology company specializing in diagnostic systems and diabetes management, announced today the successful completion of a key pre-pivotal clinical trial in Europe for its innovative continuous glucose monitoring (CGM) technology. This marks a significant milestone following the acquisition of Waveform Technologies' assets in January 2024.
The trial evaluated the enhanced CGM technology's analytical performance, which is part of Trinity Biotech's research and development efforts to improve the device's accuracy and user experience. John Gillard, President and CEO of Trinity Biotech, stated that the positive trial results confirm the effectiveness of the R&D plan and will inform further sensor design optimizations.
The updated CGM device features a reduced biosensor wire length, aimed

In [1]:
# 중국꼬
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import mysql.connector

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# MySQL 연결 설정
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}

# MySQL에 연결
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# 크롬 드라이버 경로 설정
driver = webdriver.Chrome(options=chrome_options)

# 1. 뉴스 검색 결과 페이지 접속
url = "https://www.so.com/s?q=%D2%DA%D4%AA&ie=gbk&src=zz_www_ce_cn&site=ce.cn&rg=1"
driver.get(url)
time.sleep(6)  # 페이지 로딩 대기

# 2. 뉴스 목록에서 링크 추출
news_links = set()
news_list = driver.find_elements(By.CSS_SELECTOR, "#main ul li a")

for news in news_list:
    href = news.get_attribute("href")
    if href:
        news_links.add(href)

# 링크 리스트 확인
print(f"총 {len(news_links)}개의 뉴스를 발견했습니다.")

# 3. 각 뉴스 페이지에 접속하여 제목과 내용을 크롤링 후 MySQL에 삽입
for link in news_links:
    driver.get(link)
    time.sleep(2)  # 페이지 로딩 대기

    try:
        # 뉴스 제목 추출
        title = driver.find_element(By.CSS_SELECTOR, "#articleTitle").text
        print(f"Title: {title}")

        # 뉴스 내용 추출 (id="article_p" 내의 모든 <p> 태그를 순회하여 텍스트 추출)
        content_elements = driver.find_elements(By.CSS_SELECTOR, "#article_p .TRS_Editor p")
        content = "\n".join([element.text for element in content_elements])
        
        # 4. MySQL에 데이터 삽입
        sql_insert = """
            INSERT INTO fo_news_table (title, content, state, url) 
            VALUES (%s, %s, %s, %s)
        """
        data = (title, content, 'CNY', link)  # state는 'CNY'로 고정
        cursor.execute(sql_insert, data)
        conn.commit()  # 변경사항 저장

    except Exception as e:
        print(f"크롤링 실패: {e}")

# 드라이버 종료
driver.quit()

# MySQL 연결 종료
cursor.close()
conn.close()

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))